Inital thoughts:
Population estimate table: can fill missing values in 0-4 year olds by taking the sum of 1-4, 5-9, and 10-14 from the estimate of 0-14 - this gives us the number of 0 year olds, which can be added to the 1-4 yos.

Migration table: Can fill missing values in net migration by subtracting emigrants from immigrants. In earlier years the totals aren't calculated, but can be derived from summing the completed ages

Can we look at births/deaths as a weibull distribution modeling time to failure with historic infant mortality rates as an estimate of the rate parameter?

Population (https://data.cso.ie/table/PEA01) and migration (https://data.cso.ie/table/PEA03) datasets were retrieved from data.cso.ie at 10AM on the 6th of October 2023

In [1]:
import pandas as pd
import seaborn as sns

Our first dataset is the population estimates in April from 1950 to 2023
Load the population dataframe and change the column names to snake_case by converting to lowercase and replacing spaces with underscores.

In [2]:
population_df = pd.read_csv('data/population_estimates_1950_2023.csv')
population_df = population_df.rename(columns={x: x.lower().replace(' ', '_') for x in population_df.columns})
print(population_df.dtypes)
population_df.head(15)

statistic_label     object
year                 int64
age_group           object
sex                 object
unit                object
value              float64
dtype: object


,statistic_label,year,age_group,sex,unit,value
0,Population Estimates (Persons in April),1950,Under 1 year,Both sexes,Thousand,61.1
1,Population Estimates (Persons in April),1950,Under 1 year,Male,Thousand,31.4
2,Population Estimates (Persons in April),1950,Under 1 year,Female,Thousand,29.7
3,Population Estimates (Persons in April),1950,0 - 4 years,Both sexes,Thousand,NaN
4,Population Estimates (Persons in April),1950,0 - 4 years,Male,Thousand,NaN
5,Population Estimates (Persons in April),1950,0 - 4 years,Female,Thousand,NaN
6,Population Estimates (Persons in April),1950,0 - 14 years,Both sexes,Thousand,851.2
7,Population Estimates (Persons in April),1950,0 - 14 years,Male,Thousand,434.6
8,Population Estimates (Persons in April),1950,0 - 14 years,Female,Thousand,416.6
9,Population Estimates (Persons in April),1950,1 - 4 years,Both sexes,Thousand,249.1


We can see that we have missing values the value column, however, this only occurs in the 0-4 years age groups, not every year, and only in years until 1995

In [3]:
population_df.loc[pd.isna(population_df.value)]['age_group'].value_counts()

age_group
0 - 4 years    114
Name: count, dtype: int64

In [4]:
population_df.loc[(population_df['age_group'] == "0 - 4 years") & (pd.isna(population_df['value']))].head()

,statistic_label,year,age_group,sex,unit,value
3,Population Estimates (Persons in April),1950,0 - 4 years,Both sexes,Thousand,NaN
4,Population Estimates (Persons in April),1950,0 - 4 years,Male,Thousand,NaN
5,Population Estimates (Persons in April),1950,0 - 4 years,Female,Thousand,NaN
165,Population Estimates (Persons in April),1952,0 - 4 years,Both sexes,Thousand,NaN
166,Population Estimates (Persons in April),1952,0 - 4 years,Male,Thousand,NaN


In [5]:
population_df.loc[(population_df['age_group']=="0 - 4 years") & (pd.isna(population_df['value']))].tail()

,statistic_label,year,age_group,sex,unit,value
3568,Population Estimates (Persons in April),1994,0 - 4 years,Male,Thousand,NaN
3569,Population Estimates (Persons in April),1994,0 - 4 years,Female,Thousand,NaN
3648,Population Estimates (Persons in April),1995,0 - 4 years,Both sexes,Thousand,NaN
3649,Population Estimates (Persons in April),1995,0 - 4 years,Male,Thousand,NaN
3650,Population Estimates (Persons in April),1995,0 - 4 years,Female,Thousand,NaN


We can try to fill in the missing 0-4 year olds using a combination of other columns;
We have "Under 1 year" as a value within age_group, so we should be able to add this to the "1 - 4 years" value to get "0 - 4 years".
We can check if this works by comparing against the completed entries post-1995.

In [6]:
pivoted_population_df = population_df.pivot(columns='age_group', index=['year', 'sex'], values='value')
pivoted_population_df.head()

age_group        0 - 14 years  0 - 4 years  1 - 4 years  10 - 14 years  \
year sex                                                                 
1950 Both sexes         851.2          NaN        249.1          262.1   
     Female             416.6          NaN        121.6          128.7   
     Male               434.6          NaN        127.5          133.4   
1951 Both sexes         854.8        312.9        249.3          260.9   
     Female             418.4        152.6        121.7          128.2   

age_group        15 - 19 years  15 - 24 years  15 years and over  \
year sex                                                           
1950 Both sexes          244.0          452.6             2117.8   
     Female              117.4          217.7             1042.4   
     Male                126.6          234.9             1075.4   
1951 Both sexes          241.2          443.4             2105.8   
     Female              115.4          212.2             1035.6   

age_group        20 - 24 years  25 - 29 years  25 - 44 years  ...  \
year sex                                                      ...   
1950 Both sexes          208.6          201.1          773.6  ...   
     Female              100.3          100.2          380.2  ...   
     Male                108.3          100.9          393.4  ...   
1951 Both sexes          202.2          198.4          771.2  ...   
     Female               96.8           98.8          378.9  ...   

age_group        55 - 59 years  60 - 64 years  65 - 69 years  \
year sex                                                       
1950 Both sexes          130.4          121.9          109.3   
     Female               64.6           60.6           54.1   
     Male                 65.8           61.3           55.2   
1951 Both sexes          128.8          122.1          107.5   
     Female               63.8           60.7           53.4   

age_group        65 years and over  70 - 74 years  75 - 79 years  \
year sex                                                           
1950 Both sexes              317.1          100.4           64.0   
     Female                  161.3           50.9           32.6   
     Male                    155.8           49.5           31.4   
1951 Both sexes              316.4          100.1           64.6   
     Female                  161.4           51.0           32.9   

age_group        80 - 84 years  85 years and over  All ages  Under 1 year  
year sex                                                                   
1950 Both sexes           30.1               13.3    2969.0          61.1  
     Female               15.9                7.8    1459.0          29.7  
     Male                 14.2                5.5    1510.0          31.4  
1951 Both sexes           30.9               13.3    2960.6          63.6  
     Female               16.2                7.8    1454.0          30.9  

[5 rows x 27 columns]

In [7]:
pivoted_population_df['candidate_imputation'] = pivoted_population_df['Under 1 year'] + pivoted_population_df['1 - 4 years']
nonnull_rows = pivoted_population_df.loc[~pd.isna(pivoted_population_df['0 - 4 years'])]
print((nonnull_rows['0 - 4 years'] == nonnull_rows['candidate_imputation']).value_counts())
print(len(nonnull_rows))

True     61
False    47
Name: count, dtype: int64
108


We see however that this doesn't work, as our candidate value does not equal the reported value in 47/108 cases.

In [8]:
discrepancy_rows = nonnull_rows[nonnull_rows['0 - 4 years'] != nonnull_rows['candidate_imputation']].copy()
discrepancy_rows['difference'] = (discrepancy_rows['0 - 4 years'] - discrepancy_rows['candidate_imputation']).map('{:,.2f}'.format)
discrepancy_rows[['1 - 4 years', 'Under 1 year', '0 - 4 years', 'candidate_imputation', 'difference']]

age_group        1 - 4 years  Under 1 year  0 - 4 years  candidate_imputation  \
year sex                                                                        
1951 Both sexes        249.3          63.6        312.9                 312.9   
     Male              127.6          32.6        160.0                 160.2   
1961 Female            116.7          30.6        147.4                 147.3   
1966 Both sexes        252.3          63.6        315.9                 315.9   
1971 Both sexes        250.8          64.9        315.6                 315.7   
     Male              128.6          33.3        161.8                 161.9   
1979 Female            132.7          35.2        167.8                 167.9   
     Male              139.5          36.2        175.6                 175.7   
1981 Female            136.3          35.7        171.9                 172.0   
1986 Both sexes        262.9          61.2        324.1                 324.1   
1991 Female            107.5          25.7        133.8                 133.2   
1997 Female             96.9          24.4        121.3                 121.3   
1998 Both sexes        201.1          52.6        253.6                 253.7   
1999 Both sexes        206.4          53.3        259.8                 259.7   
     Female             99.9          25.9        125.8                 125.8   
     Male              106.5          27.4        134.0                 133.9   
2000 Both sexes        212.5          53.6        266.2                 266.1   
2001 Female            106.1          26.7        133.0                 132.8   
2006 Both sexes        241.3          61.1        302.3                 302.4   
     Female            117.9          29.8        147.7                 147.7   
     Male              123.4          31.3        154.7                 154.7   
2007 Female            118.8          32.2        151.1                 151.0   
2008 Male              129.1          36.9        165.9                 166.0   
2009 Both sexes        261.7          73.6        335.2                 335.3   
     Male              133.8          37.7        171.4                 171.5   
2010 Both sexes        272.0          74.9        347.0                 346.9   
     Female            133.0          36.5        169.4                 169.5   
2011 Both sexes        283.6          72.5        356.0                 356.1   
2012 Female            280.7          35.4        175.4                 316.1   
     Male              145.5          37.0        182.7                 182.5   
2013 Female            277.3          33.5        173.1                 310.8   
     Male              144.9          35.0        180.0                 179.9   
2014 Both sexes        278.4          66.9        345.4                 345.3   
     Female            278.7          32.6        169.3                 311.3   
     Male              141.7          34.0        176.0                 175.7   
2015 Both sexes        273.8          64.2        337.9                 338.0   
     Female            279.7          31.4        165.6                 311.1   
     Male              139.6          33.0        172.4                 172.6   
2016 Female            281.9          30.4        161.7                 312.3   
     Male              137.8          32.0        169.7                 169.8   
2017 Male              133.7          32.4        166.2                 166.1   
2018 Male              132.2          31.7        163.9                 163.9   
2020 Female            121.6          28.3        150.0                 149.9   
     Male              128.9          29.8        158.8                 158.7   
2021 Female            119.3          27.4        146.8                 146.7   
2022 Both sexes        238.2          57.7        295.8                 295.9   
     Male              122.0          29.5        151.6                 151.5   

age_group       difference  
y

Some of these discrepancises appear to arise from floating point arithmetic, such as "Both sexes" in 1951, where the difference is <<0.1.

A lot of these may be rounding errors arising from aggregation from a lower level, as many values are off by small but nonnegligable amounts (0.1-0.6).

However these discrepancies also draw our attention to some strange values - for example, in 2015, the number of Female 1-4 year olds was greater than the value for both sexes, which doesn't make much sense.

In [9]:
pivoted_population_df[['1 - 4 years', 'Under 1 year', '0 - 4 years', '0 - 14 years', 'candidate_imputation']].loc[2015]

age_group,1 - 4 years,Under 1 year,0 - 4 years,0 - 14 years,candidate_imputation
sex,,,,,
Both sexes,273.8,64.2,337.9,1001.7,338.0
Female,279.7,31.4,165.6,490.0,311.1
Male,139.6,33.0,172.4,511.8,172.6


We can look at this in the original data just to confirm that we don't have a misspecified pivot or other programming error. 

I thought it might be possible that the unit in this case was 'Hundred' instead of "Thousand" and that's what was throwing things off, but that doesn't seem to be the case, and this may be a genuine reporting error.

In [10]:
population_df[(population_df['year'] == 2015) & (population_df['age_group'] == '1 - 4 years')]

,statistic_label,year,age_group,sex,unit,value
5274,Population Estimates (Persons in April),2015,1 - 4 years,Both sexes,Thousand,273.8
5275,Population Estimates (Persons in April),2015,1 - 4 years,Male,Thousand,139.6
5276,Population Estimates (Persons in April),2015,1 - 4 years,Female,Thousand,279.7


These large discrepancies (>1000 people) only appear to occur in Females between 2012 and 2016 - the years between the 2011 and 2016 census.

There are multiple possibilities for what's going on:

- The number of female 1-4 year olds is overreported (this seems to be the most likely as, in 2015, it is so much higher than the values for 0-4 years and is over
- The number for "Both sexes" is underreported, and should be taken as the sum of the reported Male and Female figures
- Neither figure is correct

The only one of these years that contained a census was 2016, so we can use this as an alternative source to check. The age breakdown for the 2016 census is found at - https://data.cso.ie/table/E3018 (retrieved at 2pm on the 6th of October 2023)

In [11]:
census_2016_age_df = pd.read_csv('data/census_2016_age_profile.csv')
census_2016_age_df = census_2016_age_df.rename(columns={x: x.lower().replace(' ', '_') for x in census_2016_age_df.columns})
census_2016_age_df.head()

,statistic_label,censusyear,electoral_division,unit,value
0,Persons of all ages,2016,State,Number,4761865
1,Persons of all ages,2016,"001 Carlow Urban, Co. Carlow",Number,4560
2,Persons of all ages,2016,"002 Graigue Urban, Co. Carlow",Number,1405
3,Persons of all ages,2016,"012 Ballinacarrig (Part Urban), Co. Carlow",Number,1025
4,Persons of all ages,2016,"019 Carlow Rural (Part Urban), Co Carlow",Number,14546


This data is split by electoral division, but for our verification we are primarily concerned with the total, which is given by the electoral division value of 'State'

In [12]:
census_2016_age_df.loc[census_2016_age_df['electoral_division'] == 'State']

,statistic_label,censusyear,electoral_division,unit,value
0,Persons of all ages,2016,State,Number,4761865
3442,Persons aged 0-4 years,2016,State,Number,331515
6884,Persons aged 5-12 years,2016,State,Number,548693
10326,Persons aged 13-18 years,2016,State,Number,371588
13768,Persons aged 19-24 years,2016,State,Number,331208
17210,Persons aged 25-44 years,2016,State,Number,1406291
20652,Persons aged 45-64 years,2016,State,Number,1135003
24094,Persons aged 65-69 years,2016,State,Number,211236
27536,Persons aged 70 years and over,2016,State,Number,426331


In [13]:
census_2016_age_df.loc[(census_2016_age_df['electoral_division'] == 'State') &
    (census_2016_age_df['statistic_label'] != 'Persons of all ages')]['value'].sum()

4761865

We confirm that the total value for persons of all ages is equal to the sum of the age subdivisions, so the data is internally consistent in this sense.

We notice however, that the age divisions are different in this dataset compared to the estimate dataset - we do not have a 1-4 years category, which is the variable where our issue resides. We will therefore have to sum the variables which cover the 0-4 age range in our original dataset, and compare that total to the 0-4 total in the census dataset.

We can also compare the 0-4 range itself from the original dataset.

We will look at the totals we get from Both Sexes, then by Males + Females, using a variable cover of:
"Under 1 year" + "1 - 4 years" = "0 - 4 years". In each case we need to multiply the figures from the original dataset by 1000 as they are expressed in thousands of people.

In [29]:
pivoted_2016 = pivoted_population_df.loc[2016].copy()

pivoted_2016['0 - 4 years'] = pivoted_2016['0 - 4 years']*1000
pivoted_2016['0 - 4 summed'] = (pivoted_2016['Under 1 year'] + pivoted_2016['1 - 4 years'])*1000
pivoted_2016['0 - 4 census'] = census_2016_age_df.loc[
    (census_2016_age_df['electoral_division'] == 'State') & (census_2016_age_df['statistic_label'] == 'Persons aged 0-4 years')
    ]['value'].iat[0]

In [30]:
pivoted_2016[[
    '0 - 4 years', 
    '0 - 4 summed', 
    '0 - 4 census']]

age_group,0 - 4 years,0 - 4 summed,0 - 4 census
sex,,,
Both sexes,331400.0,331400.0,331515
Female,161700.0,312300.0,331515
Male,169700.0,169800.0,331515


We can see that the Both Sexes figures for both the 0-4 years and our 0-4 summed variable agree with the census figures (a difference of 115 people), wheras the Female + Male figures for our 0-4 summed variable would give us a total of over 480k, which is far greater than the census figure, leading us to conclude that the 1-4 years data for Females has likely been reported erroneously.